# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import re

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Creating city dataframe
weather_df = pd.read_csv("../weatherpy/cities.csv")
weather_df.dropna()
weather_df.head()

"""
weather_data = pd.read_csv(weather_df)

weather_df = pd.DataFrame(weather_data)
weather_df.head()
"""

'\nweather_data = pd.read_csv(weather_df)\n\nweather_df = pd.DataFrame(weather_data)\nweather_df.head()\n'

In [3]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 10 columns):
City_ID       586 non-null int64
City          586 non-null object
Country       582 non-null object
Date          586 non-null int64
Latitude      586 non-null float64
Longitude     586 non-null float64
Cloudiness    586 non-null int64
Humidity      586 non-null int64
Max Temp      586 non-null float64
Wind Speed    586 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 45.9+ KB


In [4]:
weather_df.columns

Index(['City_ID', 'City', 'Country', 'Date', 'Latitude', 'Longitude',
       'Cloudiness', 'Humidity', 'Max Temp', 'Wind Speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

plt.savefig("Humidity_Map.png")

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

temp_file=weather_df.loc[(weather_df['Max Temp']>=74)&(weather_df['Max Temp']<=95)]
wind_file=temp_file.loc[(temp_file['Wind Speed']>=5)&(temp_file['Wind Speed']<=25)]
humidity_file=wind_file.loc[(weather_df['Humidity']<=49)]
ideal_weather=humidity_file.loc[(wind_file['Cloudiness']<=70)]
ideal_weather.head(20)

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
105,105,Yulara,AU,1579402261,-25.24,130.99,33,31,89.60,12.75
147,147,San Rafael,AR,1579402275,-34.62,-68.33,58,40,80.29,11.05
152,152,Wembley,AU,1579402275,-31.93,115.80,0,36,81.00,14.99
197,197,Geraldton,AU,1579402285,-28.77,114.60,65,32,95.00,17.22
244,244,Yeppoon,AU,1579402292,-23.13,150.73,40,32,95.00,11.41
448,448,Daru,PG,1579402338,-9.08,143.21,19,49,94.59,8.16
531,531,Alice Springs,AU,1579402358,-23.70,133.88,40,37,86.00,11.41
551,551,Deniliquin,AU,1579402362,-35.53,144.97,0,24,91.11,11.86


In [9]:
ideal_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 105 to 551
Data columns (total 10 columns):
City_ID       8 non-null int64
City          8 non-null object
Country       8 non-null object
Date          8 non-null int64
Latitude      8 non-null float64
Longitude     8 non-null float64
Cloudiness    8 non-null int64
Humidity      8 non-null int64
Max Temp      8 non-null float64
Wind Speed    8 non-null float64
dtypes: float64(4), int64(4), object(2)
memory usage: 704.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -25.2390223,
                    "lng": 130.9834051
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.23774217010728,
                        "lng": 130.9849756798928
                    },
                    "southwest": {
                        "lat": -25.24044182989272,
                        "lng": 130.9822760201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "bed6ce3fd54436f7dc5bf1b682a33d64a3bd2d61",
            "name": "Sails in the Desert",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2738,
                    "html_attributions": [
                 

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAN5VqEXWKoVo2Bmof6LF0AahKAu9QtDOD_ECZL_p6KONqg6J9t5yPLWlMp413A39cJZdDAjq9GJANRjP3LjjBFbVKAItJbph2tdmgXGD7i2diJPcNkd5BQdAm3TitgnZDB_CPunnxO8zgRYJ7ynsbvz4A_mj2NWn2lFgG9qaC6WSz7Y0WIvT4tALHzgAsmcExL2UVLGBcG5ZZDO-XdT0HFLpGhtDaLmHKjGKHOaZwPMUK_kGBPRgFhRQnrq5jLiKPlq6cx2xKSU-UHcgQQa8bDVVtAI6tJB6B9CEqItE6EAf4s9GbkSUhHJikK4ekn0QJHIOvP1ceZNL7PaA1Bf3nl7D5wvNpi8r8AgR2_QGxGbsG2ftrrvPFb7QScTN4wquXq3v-yBPZE5ltmm5koYO9JMSEHyOAMJmtJuKZ0L_JjhW0fMaFOGnkffJ9bE4kEugsJi22ieUBOeH",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.646365,
                    "lng": -68.462169
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.64473257010728,
                        "lng": -68.46068022010728
                    },
                    "southwest": {
                        "lat": -34.64743222989272,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAAmIKRx7y-Tcs5B7vVU7UcukdZ0ZH8zqpGwM_q4TYtbGhL7tX1NliooMthv5JU2u3BFrYof60TqsorZMO0aNf68L8YfHKN8M8VbLwlw9aBLiX6BwwVWwuYLFQjTrSuy3Z4yEPVvrMdKHf5UT5UK1A0wma_KUnW3t6ls0_647meFS0HPt6Q8I_4c8p3c67cvO5Scz1PlGWasfINoZMdmDUzrowdH4l8BityYF6REksWGsrIt3VlCJC12dkf3AtS082y-U1xtPoWsZPfoa4I1hZ02xSpBltHENgNoOwUt_TfhkmwLTDuWfl90q74SRgZp9GdnhM4UVohYA6zEOCXs0imG_-jOV3bkwQ7WVpgjA4BOmhrnM96dc6bKK2gahOpInmm9g1ioZfm-ap9WsnI65vb8SED6_hk18jfLE0TsvK6yTXB0aFJuDO-4Xcnf7R3nLzIhkeYPKo7Ty",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.932549,
                    "lng": 115.930776
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.93124322010728,
                        "lng": 115.9318429298928
                    },
                    "southwest": {
                        "lat": -31.93394287989273,
                     

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAGPIMnkOL9tP9GNWuUmH_Bdl8gGDdRKtlb8zCK7i_H7BRsoyDW46l3e7a0ncgaxAyeGWABef9M1R2oI7-j0xkvACzlEmbJir-2udxw9oGPHhirtSdyN1EJnVVrMHrCS1KrylTvazHVMO9wEvfFV0toYviYHD-0tKr8rPTT8Hr6-h7g8rOjl3D2Vsg2bMz-sBYcX8RqphZTtLnH7_vYfTFPUDbe1Lc23UFWBgLb2EUAmLngkDG_JiF9AZ9QgBwRzNwNjh2kNfQoR2mWER9SUH5S7uGged0SmiSOW_Tt5t1A2Fb7YDUfYVvcP6r5Wco_03iXa3WVvB81AdGbYKkUnLc_pbqI7yqN1K27KMbaiYm-VxatwRm2h7Qg7Bdh74BGXkJkmmpSIg1PdZgMlqVc2FJPYSEMLGggYA73sK1d4YohY42hgaFOpPmHV8BRokhZvu6c2MAx7uxmMW",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -28.8011629,
                    "lng": 114.619951
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.79970962010728,
                        "lng": 114.6216360298928
                    },
                    "southwest": {
                        "lat": -28.80240927989272,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAACkziK8MiPtsr24lPsYGMO_n22k6Z-D9g_hfhkpLBUnnDbj6AvjfcGaLGcDsoluwSNRY53SEfgqMBlIfb3iU1bd3dEaHakO2rsD-ToA3mnILUPYVTxfmNt-kkjuJH0lavOz0xfLrir9oh07RI9Q9WUY6II7QJqJkMVHBOeufqiFLKgEAVR_9FthI6bNzbTgxm_wt5jEgU5nMUgiJEYt8iW5O1Fow2vPVZC9N5wlkiHXjKjjSUANIy6v1JjfmgYrTHyLt-tkdkuyPw1TP4JWcNI03UHhLCW4F162cJ_fU-n1zaVdNy94PY4YwpFuENDqyD-4-n8H8tGpn0orRxW5VxEEnTvUubs5gn6gvkq3nG_rW3gjW5Q7PJ5nswpqca-L3Kk5a-K0uHd09sNkyK3dgt_cSEOPd0QqNYn9m4sGvbxrPSiUaFPAnHO6-5wuBIdVEuhvhDiqULgju",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -23.2887003,
                    "lng": 150.5114061
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.28730447010728,
                        "lng": 150.5127097798927
                    },
                    "southwest": {
                        "lat": -23.29000412989272,
                   

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -9.0685297,
                    "lng": 143.209553
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.067179870107276,
                        "lng": 143.2109028298927
                    },
                    "southwest": {
                        "lat": -9.06987952989272,
                        "lng": 143.2082031701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "4194dbc9ac210a40c9dca0a2095ed8612ef7535d",
            "name": "Daru hotel",
            "photos": [
                {
                    "height": 3264,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/112242827875754506078\">A Google Use

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAO7Rp9hysQ0lwIRS9mm5PCpGJi6GTVUE44lYjapytWgnbHFM2ERvAFPbi13fhlt6Lfcf7EFAj-YCrFlWrfKCoYkAf2EN0ZYYgwI1jIwtXlGSCy9EPAbmfDvTjCSIH-4puiaDSyUnMxbkKka7_Lt9H_q_B4YRpwEYLvLinjN56rrkRj_-wCEbT6JiLK0wDNDhd5lnD-7BNXYN3hCOuYRSfrJcy29QXhi7O4qu_dxdTcFEhmZ2HNKFGv8rSXFG1CRcmmejCBLnehb3JCZyvP-8_jUR6TAIP-UZ55AHO8jCw8KjyiMrKUVtndRSj8WTTcof-Zf79Z9Dj51cAqgEKnvowiscFOLxYWWrD1DleAxYjg3H7ibIHobIYAenVJzRFPeIWJEVrjvbRe14uFq7zjGlmr8SENDbZU0gxm5ieGE_RZ1IZEUaFCQqJg33twjEkOe6sq4Pm87kLeyK",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -35.5378212,
                    "lng": 144.9656212
                },
                "viewport": {
                    "northeast": {
                        "lat": -35.53638747010728,
                        "lng": 144.9671083798927
                    },
                    "southwest": {
                        "lat": -35.53908712989272,
                   

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,105,Yulara,AU,1579402261,-25.24,130.99,33,31,89.60,12.75,Sails in the Desert
1,147,San Rafael,AR,1579402275,-34.62,-68.33,58,40,80.29,11.05,Algodon Wine Estates
2,152,Wembley,AU,1579402275,-31.93,115.80,0,36,81.00,14.99,International on the Water Hotel
3,197,Geraldton,AU,1579402285,-28.77,114.60,65,32,95.00,17.22,ibis Styles Geraldton
4,244,Yeppoon,AU,1579402292,-23.13,150.73,40,32,95.00,11.41,Korte's Resort
5,448,Daru,PG,1579402338,-9.08,143.21,19,49,94.59,8.16,Daru hotel
6,531,Alice Springs,AU,1579402358,-23.70,133.88,40,37,86.00,11.41,DoubleTree by Hilton Hotel Alice Springs
7,551,Deniliquin,AU,1579402362,-35.53,144.97,0,24,91.11,11.86,Deniliquin Motel


In [11]:
hotel_df.head(15)

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,105,Yulara,AU,1579402261,-25.24,130.99,33,31,89.60,12.75,Sails in the Desert
1,147,San Rafael,AR,1579402275,-34.62,-68.33,58,40,80.29,11.05,Algodon Wine Estates
2,152,Wembley,AU,1579402275,-31.93,115.80,0,36,81.00,14.99,International on the Water Hotel
3,197,Geraldton,AU,1579402285,-28.77,114.60,65,32,95.00,17.22,ibis Styles Geraldton
4,244,Yeppoon,AU,1579402292,-23.13,150.73,40,32,95.00,11.41,Korte's Resort
5,448,Daru,PG,1579402338,-9.08,143.21,19,49,94.59,8.16,Daru hotel
6,531,Alice Springs,AU,1579402358,-23.70,133.88,40,37,86.00,11.41,DoubleTree by Hilton Hotel Alice Springs
7,551,Deniliquin,AU,1579402362,-35.53,144.97,0,24,91.11,11.86,Deniliquin Motel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
fig=gmaps.figure(map_type="TERRAIN")
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

plt.savefig("Heat_Map.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>